In [25]:
from pymongo import MongoClient

In [26]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='/home/apprenant/Documents/DevIA/Projet_DevIA/Scrapping-IMDB/.env')
ATLAS_KEY = os.getenv('ATLAS_KEY')
client = MongoClient(ATLAS_KEY)
db = client['IMBDScrapping']
collection = db['Movies']

In [24]:
longest_movie = collection.find_one(sort=[('time', -1)], projection={'title': 1, 'time': 1})
print(longest_movie)

OperationFailure: bad auth : authentication failed, full error: {'ok': 0, 'errmsg': 'bad auth : authentication failed', 'code': 8000, 'codeName': 'AtlasError'}